In [593]:
import requests
import pandas as pd

In [594]:
def download_esios_id(id, fecha_ini, fecha_fin, agrupacion):
                       
    cab = dict()
    #cab ['x-api-key'] = st.secrets['ESIOS_API_KEY']
    
    cab ['x-api-key'] = "496b263791ef0dcaf80b803b47b332a13b01f2c2352e018b624c7a36a0eaffc0"
    url_id = 'https://api.esios.ree.es/indicators'
    url = f'{url_id}/{id}?geo_ids[]=3&start_date={fecha_ini}T00:00:00&end_date={fecha_fin}T23:59:59&time_trunc={agrupacion}'
    print(url)
    datos_origen = requests.get(url, headers = cab).json()
    
    #arreglamos los datos origen    
    datos =pd.DataFrame(datos_origen['indicator']['values'])
    datos = (datos
        .assign(datetime=lambda vh_: pd #formateamos campo fecha, desde un str con diferencia horaria a un naive
            .to_datetime(vh_['datetime'],utc=True)  # con la fecha local
            .dt
            .tz_convert('Europe/Madrid')
            .dt
            .tz_localize(None)
        )   
        .loc[:,['datetime','value']]
    )
    datos['fecha'] = datos['datetime'].dt.date
    datos['fecha'] = pd.to_datetime(datos['fecha'], format = '%d/%m/%Y')
    datos['año'] = datos['datetime'].dt.year
    datos['mes'] = datos['datetime'].dt.month
    datos['dia'] = datos['datetime'].dt.day
    datos['hora'] = datos['datetime'].dt.hour
    meses_español = {1: "ene", 2: "feb", 3: "mar", 4: "abr", 5: "may", 6: "jun",
                 7: "jul", 8: "ago", 9: "sep", 10: "oct", 11: "nov", 12: "dic"}
    datos.rename(columns={'value':'spot'}, inplace=True)
    datos['mes_nombre'] = datos['mes'].map(meses_español)
    #datos = datos['fecha', 'año', 'mes', 'mes_nombre', 'dia', 'hora', 'spot' ]
    datos.set_index('datetime', inplace = True)
    
    

    
    return datos

In [595]:
fecha_ini_spot = '2023-01-01'
fecha_fin_spot = '2025-12-31'
id = 600
agrupacion = 'hour'
df_datos_spot = download_esios_id(id, fecha_ini_spot, fecha_fin_spot, agrupacion)
df_datos_spot

https://api.esios.ree.es/indicators/600?geo_ids[]=3&start_date=2023-01-01T00:00:00&end_date=2025-12-31T23:59:59&time_trunc=hour


,spot,fecha,año,mes,dia,hora,mes_nombre
datetime,,,,,,,
2023-01-01 00:00:00,0.00,2023-01-01,2023,1,1,0,ene
2023-01-01 01:00:00,0.00,2023-01-01,2023,1,1,1,ene
2023-01-01 02:00:00,0.00,2023-01-01,2023,1,1,2,ene
2023-01-01 03:00:00,0.00,2023-01-01,2023,1,1,3,ene
2023-01-01 04:00:00,0.00,2023-01-01,2023,1,1,4,ene
...,...,...,...,...,...,...,...
2025-03-06 19:00:00,126.02,2025-03-06,2025,3,6,19,mar
2025-03-06 20:00:00,117.01,2025-03-06,2025,3,6,20,mar
2025-03-06 21:00:00,105.51,2025-03-06,2025,3,6,21,mar


In [596]:
df_periodos = pd.read_excel('periodos_horarios.xlsx', parse_dates = ['fecha'])
df_periodos['fecha'] = pd.to_datetime(df_periodos['fecha'], format = '%d/%m/%Y', errors = 'coerce')
df_periodos

,fecha,año,mes,dia,hora,dh_3p,dh_6p
0,2023-01-01,2023,1,1,0,P3,P6
1,2023-01-01,2023,1,1,1,P3,P6
2,2023-01-01,2023,1,1,2,P3,P6
3,2023-01-01,2023,1,1,3,P3,P6
4,2023-01-01,2023,1,1,4,P3,P6
...,...,...,...,...,...,...,...
26299,2025-12-31,2025,12,31,19,P1,P1
26300,2025-12-31,2025,12,31,20,P1,P1
26301,2025-12-31,2025,12,31,21,P1,P1
26302,2025-12-31,2025,12,31,22,P2,P2


In [597]:
df_datos = df_datos_spot.copy()
df_datos = df_datos.merge(df_periodos[['fecha', 'hora', 'dh_3p', 'dh_6p']], on = ['fecha', 'hora'], how = 'left')
orden_col = ['fecha', 'año', 'mes', 'mes_nombre', 'dia', 'hora', 'dh_3p', 'dh_6p', 'spot']
df_datos = df_datos[orden_col]
df_datos

,fecha,año,mes,mes_nombre,dia,hora,dh_3p,dh_6p,spot
0,2023-01-01,2023,1,ene,1,0,P3,P6,0.00
1,2023-01-01,2023,1,ene,1,1,P3,P6,0.00
2,2023-01-01,2023,1,ene,1,2,P3,P6,0.00
3,2023-01-01,2023,1,ene,1,3,P3,P6,0.00
4,2023-01-01,2023,1,ene,1,4,P3,P6,0.00
...,...,...,...,...,...,...,...,...,...
19099,2025-03-06,2025,3,mar,6,19,P1,P2,126.02
19100,2025-03-06,2025,3,mar,6,20,P1,P2,117.01
19101,2025-03-06,2025,3,mar,6,21,P1,P2,105.51
19102,2025-03-06,2025,3,mar,6,22,P2,P3,91.96


In [598]:
def download_esios_ids(indicadores, fecha_inicio, fecha_fin, time_trunc = 'hour'):
    """
    Descarga datos esios desde un determinado identidficador y entre dos fechas
    
    Parameters
    ----------
    token : str
        El token de esios necesario para realizar las llamadas al API
    
    indicadores : list
        Lista con los strings de los indicadores de los que queremos bajar datos
        
    fecha_inicio : str
        Fecha con formato %Y-%M-%d, que indica la fecha desde la que se quiere bajar los datos.
        Ejemplo 2022-10-30, 30 Octubre de 2022.
    
    fecha_fin : str
        Fecha con formato %Y-%M-%d, que indica la fecha hasta la que se quiere bajar los datos.
        Ejemplo 2022-10-30, 30 Octubre de 2022.
        
    time_trunc : str, optional
        Campo adicional que nos permite elegir la granularidad de los datos que queremos bajar.
        
    Returns
    -------
    DataFrame
        Dataframe de pandas con los datos solicitados
    
    """
    
    # preparamos la cabecera a insertar en la llamada. Vease la necesidad de disponer el token de esios
    
    cab = dict()
    
    cab ['x-api-key'] = st.secrets['ESIOS_API_KEY']
    
    # preparamos la url básica a la que se le añadiran los campos necesarios 
    
    end_point = 'https://api.esios.ree.es/indicators'
    
    # El procedimiento es sencillo: 
    # a) por cada uno de los indicadores configuraremos la url, según las indicaciones de la documentación.
    # b) Hacemos la llamada y recogemos los datos en formato json.
    # c) Añadimos la información a una lista
    
    lista=[]

    for indicador in indicadores:
        url = f'{end_point}/{indicador}?start_date={fecha_inicio}T00:00:00&end_date={fecha_fin}T23:59:59&time_trunc={time_trunc}'
        print (url)
        response = requests.get(url, headers=cab).json()
        lista.append(pd.json_normalize(data=response['indicator'], record_path=['values'], meta=['id','name','short_name'], errors='ignore'))

    # Devolvemos como salida de la función un df fruto de la concatenación de los elemenos de la lista
    # Este procedimiento, con una sola concatenación al final, es mucho más eficiente que hacer múltiples 
    # concatenaciones.
    
    return pd.concat(lista, ignore_index=True )

In [599]:
# 793: RRTT PBF RT3
# 794: RRTT Tiempo Real RT6
# 798: Banda Secundaria BS3
# 799: Desvíos medidos EXD
# 800: Saldo de desvíos DSV
# 802: Saldo P.O.14.6 IN7
# 1285: Control Factor de Potencia CFP
# 1366: Incumplimiento energía de balance BALX
 
# 803: No usado
# 797: No usado
# 1276 SI - No usado

ids_ssaa = [793, 794, 798, 799, 800, 802, 1285, 1366]

In [600]:
df_ssaa_origen = download_esios_ids(ids_ssaa, fecha_ini_spot, fecha_fin_spot, 'hour' )
df_ssaa_origen

https://api.esios.ree.es/indicators/793?start_date=2023-01-01T00:00:00&end_date=2025-12-31T23:59:59&time_trunc=hour
https://api.esios.ree.es/indicators/794?start_date=2023-01-01T00:00:00&end_date=2025-12-31T23:59:59&time_trunc=hour
https://api.esios.ree.es/indicators/798?start_date=2023-01-01T00:00:00&end_date=2025-12-31T23:59:59&time_trunc=hour
https://api.esios.ree.es/indicators/799?start_date=2023-01-01T00:00:00&end_date=2025-12-31T23:59:59&time_trunc=hour
https://api.esios.ree.es/indicators/800?start_date=2023-01-01T00:00:00&end_date=2025-12-31T23:59:59&time_trunc=hour
https://api.esios.ree.es/indicators/802?start_date=2023-01-01T00:00:00&end_date=2025-12-31T23:59:59&time_trunc=hour
https://api.esios.ree.es/indicators/1285?start_date=2023-01-01T00:00:00&end_date=2025-12-31T23:59:59&time_trunc=hour
https://api.esios.ree.es/indicators/1366?start_date=2023-01-01T00:00:00&end_date=2025-12-31T23:59:59&time_trunc=hour


,value,datetime,datetime_utc,tz_time,geo_id,geo_name,id,name,short_name
0,15.16,2023-01-01T00:00:00.000+01:00,2022-12-31T23:00:00Z,2022-12-31T23:00:00.000Z,8741,Península,793,Precio medio horario componente restricciones ...,Com. Libre
1,15.69,2023-01-01T01:00:00.000+01:00,2023-01-01T00:00:00Z,2023-01-01T00:00:00.000Z,8741,Península,793,Precio medio horario componente restricciones ...,Com. Libre
2,26.31,2023-01-01T02:00:00.000+01:00,2023-01-01T01:00:00Z,2023-01-01T01:00:00.000Z,8741,Península,793,Precio medio horario componente restricciones ...,Com. Libre
3,27.93,2023-01-01T03:00:00.000+01:00,2023-01-01T02:00:00Z,2023-01-01T02:00:00.000Z,8741,Península,793,Precio medio horario componente restricciones ...,Com. Libre
4,28.96,2023-01-01T04:00:00.000+01:00,2023-01-01T03:00:00Z,2023-01-01T03:00:00.000Z,8741,Península,793,Precio medio horario componente restricciones ...,Com. Libre
...,...,...,...,...,...,...,...,...,...
152827,0.00,2025-03-06T19:00:00.000+01:00,2025-03-06T18:00:00Z,2025-03-06T18:00:00.000Z,8741,Península,1366,Precio medio horario componente incumplimiento...,Incumplimiento energía balance CL
152828,0.00,2025-03-06T20:00:00.000+01:00,2025-03-06T19:00:00Z,2025-03-06T19:00:00.000Z,8741,Península,1366,Precio medio horario componente incumplimiento...,Incumplimiento energía balance CL
152829,0.00,2025-03-06T21:00:00.000+01:00,2025-03-06T20:00:00Z,2025-03-06T20:00:00.000Z,8741,Península,1366,Precio medio horario componente incumplimiento...,Incumplimiento energía balance CL
152830,0.00,2025-03-06T22:00:00.000+01:00,2025-03-06T21:00:00Z,2025-03-06T21:00:00.000Z,8741,Península,1366,Precio medio horario componente incumplimiento...,Incumplimiento energía balance CL


In [601]:
df_ssaa = (df_ssaa_origen
        .assign(datetime=lambda vh_: pd #formateamos campo fecha, desde un str con diferencia horaria a un naive
            .to_datetime(vh_['datetime'],utc=True)  # con la fecha local
            .dt
            .tz_convert('Europe/Madrid')
            .dt
            .tz_localize(None)
        )   
        .loc[:,['datetime','value', 'id']]
    )
df_ssaa['fecha'] = df_ssaa['datetime'].dt.date
df_ssaa['fecha'] = pd.to_datetime(df_ssaa['fecha'], format = '%d/%m/%Y')
df_ssaa['hora'] = df_ssaa['datetime'].dt.hour

In [602]:
df_ssaa

,datetime,value,id,fecha,hora
0,2023-01-01 00:00:00,15.16,793,2023-01-01,0
1,2023-01-01 01:00:00,15.69,793,2023-01-01,1
2,2023-01-01 02:00:00,26.31,793,2023-01-01,2
3,2023-01-01 03:00:00,27.93,793,2023-01-01,3
4,2023-01-01 04:00:00,28.96,793,2023-01-01,4
...,...,...,...,...,...
152827,2025-03-06 19:00:00,0.00,1366,2025-03-06,19
152828,2025-03-06 20:00:00,0.00,1366,2025-03-06,20
152829,2025-03-06 21:00:00,0.00,1366,2025-03-06,21
152830,2025-03-06 22:00:00,0.00,1366,2025-03-06,22


In [603]:
df_ssaa = df_ssaa.copy()
df_ssaa = df_ssaa.groupby(['fecha', 'hora'], as_index=False)['value'].sum()
df_ssaa.rename(columns = {'value' : 'ssaa'}, inplace = True)
df_datos = df_datos.merge(df_ssaa[['fecha', 'hora', 'ssaa']], on = ['fecha', 'hora'], how = 'left')


In [604]:
df_datos = df_datos.copy()

In [605]:
# Cargar todas las tablas del archivo (incluyendo nombres de tablas definidas en Excel)
file_path = r"C:\Users\jovid\Documents\015 JOSE\070 ENERGIA\002 HERRAMIENTAS EXCEL\FUNCIONALES\004 LUZ componentes regulados.xlsx"

In [606]:
# Cargar todas las hojas
sheets_dict = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Crear un diccionario con todas las tablas identificadas
tablas = {sheet_name: df for sheet_name, df in sheets_dict.items()}

# Acceder a cualquier tabla por su nombre de hoja
df_ppcc = tablas["PPCC"]
df_osom = tablas['OSOM']
df_perdidas_boe = tablas['PERDIDAS']
df_pycs_energia = tablas['PYC_E']

In [607]:
df_datos['ppcc_2.0'] = None
for _, row in df_ppcc.iterrows():
    mask = (df_datos["fecha"] >= row["fecha_inicio"]) & (df_datos["fecha"] <= row["fecha_final"]) & (df_datos["dh_3p"] == row["periodo"])
    df_datos.loc[mask, "ppcc_2.0"] = row["2.0TD"]
df_datos['ppcc_3.0'] = None
for _, row in df_ppcc.iterrows():
    mask = (df_datos["fecha"] >= row["fecha_inicio"]) & (df_datos["fecha"] <= row["fecha_final"]) & (df_datos["dh_6p"] == row["periodo"])
    df_datos.loc[mask, "ppcc_3.0"] = row["3.0TD"]
df_datos['ppcc_6.1'] = None    
for _, row in df_ppcc.iterrows():
    mask = (df_datos["fecha"] >= row["fecha_inicio"]) & (df_datos["fecha"] <= row["fecha_final"]) & (df_datos["dh_6p"] == row["periodo"])
    df_datos.loc[mask, "ppcc_6.1"] = row["6.1TD"]  
df_datos[['ppcc_2.0', 'ppcc_3.0', 'ppcc_6.1']] *= 1000  

In [608]:
df_datos = df_datos.copy()

In [609]:
df_datos['osom'] = None
for _, row in df_osom.iterrows():
    mask = (df_datos["fecha"] >= row["fecha_inicio"]) & (df_datos["fecha"] <= row["fecha_final"])
    df_datos.loc[mask, "osom"] = row["osom"]

In [610]:
df_datos

,fecha,año,mes,mes_nombre,dia,hora,dh_3p,dh_6p,spot,ssaa,ppcc_2.0,ppcc_3.0,ppcc_6.1,osom
0,2023-01-01,2023,1,ene,1,0,P3,P6,0.00,24.02,0.0,0.0,0.0,0.19673
1,2023-01-01,2023,1,ene,1,1,P3,P6,0.00,24.80,0.0,0.0,0.0,0.19673
2,2023-01-01,2023,1,ene,1,2,P3,P6,0.00,36.09,0.0,0.0,0.0,0.19673
3,2023-01-01,2023,1,ene,1,3,P3,P6,0.00,36.05,0.0,0.0,0.0,0.19673
4,2023-01-01,2023,1,ene,1,4,P3,P6,0.00,37.79,0.0,0.0,0.0,0.19673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19099,2025-03-06,2025,3,mar,6,19,P1,P2,126.02,2.66,0.844,0.527,0.225,0.20949
19100,2025-03-06,2025,3,mar,6,20,P1,P2,117.01,3.15,0.844,0.527,0.225,0.20949
19101,2025-03-06,2025,3,mar,6,21,P1,P2,105.51,3.09,0.844,0.527,0.225,0.20949
19102,2025-03-06,2025,3,mar,6,22,P2,P3,91.96,4.34,0.14,0.351,0.15,0.20949


In [611]:
df_datos['otros'] = 3.7

In [612]:
df_perdidas_boe

,periodo,2.0TD,3.0TD,6.1TD,6.2TD,6.3TD,6.4TD
0,P1,0.167,0.166,0.067,0.052,0.042,0.016
1,P2,0.163,0.175,0.068,0.054,0.043,0.016
2,P3,0.180,0.165,0.065,0.049,0.040,0.016
3,P4,NaN,0.165,0.065,0.050,0.040,0.016
4,P5,NaN,0.138,0.043,0.035,0.030,0.015
5,P6,NaN,0.180,0.077,0.054,0.044,0.017


In [613]:


# Asignar la columna perd_2.0_boe a df_datos según el periodo dh_3p
df_datos["perd_2.0_boe"] = df_datos["dh_3p"].map(df_perdidas_boe.set_index("periodo")["2.0TD"])
df_datos["perd_3.0_boe"] = df_datos["dh_6p"].map(df_perdidas_boe.set_index("periodo")["3.0TD"])
df_datos["perd_6.1_boe"] = df_datos["dh_6p"].map(df_perdidas_boe.set_index("periodo")["6.1TD"])


In [614]:
df_datos

,fecha,año,mes,mes_nombre,dia,hora,dh_3p,dh_6p,spot,ssaa,ppcc_2.0,ppcc_3.0,ppcc_6.1,osom,otros,perd_2.0_boe,perd_3.0_boe,perd_6.1_boe
0,2023-01-01,2023,1,ene,1,0,P3,P6,0.00,24.02,0.0,0.0,0.0,0.19673,3.7,0.180,0.180,0.077
1,2023-01-01,2023,1,ene,1,1,P3,P6,0.00,24.80,0.0,0.0,0.0,0.19673,3.7,0.180,0.180,0.077
2,2023-01-01,2023,1,ene,1,2,P3,P6,0.00,36.09,0.0,0.0,0.0,0.19673,3.7,0.180,0.180,0.077
3,2023-01-01,2023,1,ene,1,3,P3,P6,0.00,36.05,0.0,0.0,0.0,0.19673,3.7,0.180,0.180,0.077
4,2023-01-01,2023,1,ene,1,4,P3,P6,0.00,37.79,0.0,0.0,0.0,0.19673,3.7,0.180,0.180,0.077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19099,2025-03-06,2025,3,mar,6,19,P1,P2,126.02,2.66,0.844,0.527,0.225,0.20949,3.7,0.167,0.175,0.068
19100,2025-03-06,2025,3,mar,6,20,P1,P2,117.01,3.15,0.844,0.527,0.225,0.20949,3.7,0.167,0.175,0.068
19101,2025-03-06,2025,3,mar,6,21,P1,P2,105.51,3.09,0.844,0.527,0.225,0.20949,3.7,0.167,0.175,0.068
19102,2025-03-06,2025,3,mar,6,22,P2,P3,91.96,4.34,0.14,0.351,0.15,0.20949,3.7,0.163,0.165,0.065


In [615]:
url_archivos = 'https://api.esios.ree.es/archives'
cab = {'x-api-key' : '496b263791ef0dcaf80b803b47b332a13b01f2c2352e018b624c7a36a0eaffc0'}
response = requests.get(url_archivos, headers=cab)

In [616]:
# Verificar la respuesta
if response.status_code == 200:
    data_archivos = response.json()
    
    # Convertir la respuesta en un DataFrame
    df_archivos = pd.DataFrame(data_archivos["archives"])
    
    # Guardar los archivos en CSV para revisión
    #df_archivos.to_csv("archivos_ree.csv", index=False, sep=";")
    
    #print("✅ Archivo guardado: archivos_ree.csv")
    #print(df_archivos.head())  # Mostrar las primeras filas

In [639]:
import zipfile
import io
import os


#url_coefkest ='https://api.esios.ree.es/archives/40/download?date_type=datos&end_date=2025-03-31T23%3A59%3A59%2B00%3A00&locale=es&start_date=2025-01-01T00%3A00%3A00%2B00%3A00'
url_coefkest = f'https://api.esios.ree.es/archives/40/download_json?date_type=datos&end_date={fecha_fin_spot}T23:59:59&start_date={fecha_ini_spot}T00:00:00&locale=es'

cab = {'x-api-key' : '496b263791ef0dcaf80b803b47b332a13b01f2c2352e018b624c7a36a0eaffc0'}

response = requests.get(url_coefkest, headers=cab)
# Verificar el tipo de contenido de la respuesta
content_type = response.headers.get("Content-Type", "")
print(f"🔍 Content-Type recibido: {content_type}")

if "zip" in content_type or "xls" in content_type:
    # Crear una carpeta donde extraer los archivos
    extract_folder = "coef_kest_data"
    os.makedirs(extract_folder, exist_ok=True)

    # Leer el ZIP desde la respuesta
    zip_buffer = io.BytesIO(response.content)

    with zipfile.ZipFile(zip_buffer, "r") as zip_ref:
        zip_ref.extractall(extract_folder)  # Extraer todos los archivos en la carpeta
        extracted_files = zip_ref.namelist()  # Lista de archivos extraídos
    
    print(f"✅ Archivo descargado y extraído en la carpeta '{extract_folder}'")
    print(f"📂 Archivos extraídos: {extracted_files}")

else:
    print(f"❌ Error: Se recibió otro tipo de archivo ({content_type})")
    print(response.text)

🔍 Content-Type recibido: xls
✅ Archivo descargado y extraído en la carpeta 'coef_kest_data'
📂 Archivos extraídos: ['COEF_KEST_MM_202512.xls', 'COEF_KEST_MM_202511.xls', 'COEF_KEST_MM_202510.xls', 'COEF_KEST_MM_202509.xls', 'COEF_KEST_MM_202508.xls', 'COEF_KEST_MM_202507.xls', 'COEF_KEST_MM_202506.xls', 'COEF_KEST_MM_202505.xls', 'COEF_KEST_MM_202504.xls', 'COEF_KEST_MM_202503.xls', 'COEF_KEST_MM_202502.xls', 'COEF_KEST_MM_202501.xls', 'COEF_KEST_MM_202412.xls', 'COEF_KEST_MM_202411.xls', 'COEF_KEST_MM_202410.xls', 'COEF_KEST_MM_202409.xls', 'COEF_KEST_MM_202408.xls', 'COEF_KEST_MM_202407.xls', 'COEF_KEST_MM_202406.xls', 'COEF_KEST_MM_202405.xls', 'COEF_KEST_MM_202404.xls', 'COEF_KEST_MM_202403.xls', 'COEF_KEST_MM_202402.xls', 'COEF_KEST_MM_202401.xls', 'COEF_KEST_MM_202312.xls', 'COEF_KEST_MM_202311.xls', 'COEF_KEST_MM_202310.xls', 'COEF_KEST_MM_202309.xls', 'COEF_KEST_MM_202308.xls', 'COEF_KEST_MM_202307.xls', 'COEF_KEST_MM_202306.xls', 'COEF_KEST_MM_202305.xls', 'COEF_KEST_MM_202304.

In [640]:
# Carpeta donde se extrajeron los archivos
extract_folder_coefk = "coef_kest_data"
# Listar los archivos `.xls` en la carpeta
xls_files = [f for f in os.listdir(extract_folder_coefk) if f.endswith(".xls")]
# Lista para almacenar los DataFrames
df_list = []
# Leer cada archivo y almacenarlo en la lista
for file in xls_files:
    file_path = os.path.join(extract_folder_coefk, file)
    # Leer el archivo Excel
    df = pd.read_excel(file_path, skiprows=4)  # Cargar todas las hojas
    # Eliminar las filas vacías al final
    df = df.dropna(how='all')
    # Eliminar las columnas: primera, tercera y cuarta (índices 0, 2, 3)
    df = df.drop(df.columns[[0, 2, 3]], axis=1)
    # Asegurar que no haya columnas extra manteniendo solo las primeras 25 (1 de fecha + 24 de horas)
    if df.shape[1] > 25:
        df = df.iloc[:, :25]
    # Renombrar columnas
    df.columns = ['fecha'] + list(range(24))
    df_list.append(df)

# Concatenar todos los archivos en un solo DataFrame
df_coefk = pd.concat(df_list, ignore_index = True)
df_coefk['fecha'] = pd.to_datetime(df_coefk['fecha'], dayfirst=True)

# Opcional: Guardar en CSV
df_coefk.to_csv("COEF_KEST_MM_Combinado.csv", index = False, sep = ";")
print("📁 Archivo guardado: COEF_KEST_MM_Combinado.csv")

WARNING *** file size (49666) not 512 + multiple of sector size (512)
WARNING *** file size (48130) not 512 + multiple of sector size (512)
WARNING *** file size (49666) not 512 + multiple of sector size (512)
WARNING *** file size (49154) not 512 + multiple of sector size (512)
WARNING *** file size (49666) not 512 + multiple of sector size (512)
WARNING *** file size (49154) not 512 + multiple of sector size (512)
WARNING *** file size (49666) not 512 + multiple of sector size (512)
WARNING *** file size (49666) not 512 + multiple of sector size (512)
WARNING *** file size (49154) not 512 + multiple of sector size (512)
WARNING *** file size (50178) not 512 + multiple of sector size (512)
WARNING *** file size (49154) not 512 + multiple of sector size (512)
WARNING *** file size (49666) not 512 + multiple of sector size (512)
WARNING *** file size (49666) not 512 + multiple of sector size (512)
WARNING *** file size (48642) not 512 + multiple of sector size (512)
WARNING *** file siz

In [641]:
df_datos = df_datos.copy()
df_coefk.columns = df_coefk.columns.astype(str)
df_datos = df_datos.merge(df_coefk, on='fecha', how='left')
# Extraer el coeficiente correspondiente a la columna de la hora
df_datos['coef_k'] = df_datos.apply(lambda row: row[str(row['hora'])], axis=1)
columnas_horas = [str(h) for h in range(24)]  # Generamos la lista de nombres de columnas de horas
df_datos.drop(columns = columnas_horas, inplace = True)



In [642]:
df_datos

,fecha,año,mes,mes_nombre,dia,hora,dh_3p,dh_6p,spot,ssaa,...,perd_6.1,coste_2.0,coste_3.0,coste_6.1,pyc_2.0,pyc_3.0,pyc_6.1,precio_2.0,precio_3.0,precio_6.1
0,2023-01-01,2023,1,ene,1,0,P3,P6,0.00,24.02,...,0.101246,35.041877,35.041877,31.204328,3.175,2.37,1.426,38.216877,37.411877,32.630328
1,2023-01-01,2023,1,ene,1,1,P3,P6,0.00,24.80,...,0.105451,36.307322,36.307322,32.198686,3.175,2.37,1.426,39.482322,38.677322,33.624686
2,2023-01-01,2023,1,ene,1,2,P3,P6,0.00,36.09,...,0.111556,51.170703,51.170703,45.114204,3.175,2.37,1.426,54.345703,53.540703,46.540204
3,2023-01-01,2023,1,ene,1,3,P3,P6,0.00,36.05,...,0.116196,51.559309,51.559309,45.257209,3.175,2.37,1.426,54.734309,53.929309,46.683209
4,2023-01-01,2023,1,ene,1,4,P3,P6,0.00,37.79,...,0.119037,54.08609,54.08609,47.348712,3.175,2.37,1.426,57.26109,56.45609,48.774712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19099,2025-03-06,2025,3,mar,6,19,P1,P2,126.02,2.66,...,0.082511,162.879241,163.803858,145.929771,92.539,36.409,24.986,255.418241,200.212858,170.915771
19100,2025-03-06,2025,3,mar,6,20,P1,P2,117.01,3.15,...,0.081998,152.319422,153.152869,136.503746,92.539,36.409,24.986,244.858422,189.561869,161.489746
19101,2025-03-06,2025,3,mar,6,21,P1,P2,105.51,3.09,...,0.084072,138.809001,139.555604,124.045466,92.539,36.409,24.986,231.348001,175.964604,149.031466
19102,2025-03-06,2025,3,mar,6,22,P2,P3,91.96,4.34,...,0.086071,123.839028,124.369732,110.6325,28.201,17.674,11.795,152.040028,142.043732,122.4275


In [643]:
df_datos = df_datos.copy()
df_datos['perd_2.0'] = df_datos['perd_2.0_boe'] * df_datos['coef_k']
df_datos['perd_3.0'] = df_datos['perd_3.0_boe'] * df_datos['coef_k']
df_datos['perd_6.1'] = df_datos['perd_6.1_boe'] * df_datos['coef_k']
df_datos

,fecha,año,mes,mes_nombre,dia,hora,dh_3p,dh_6p,spot,ssaa,...,perd_6.1,coste_2.0,coste_3.0,coste_6.1,pyc_2.0,pyc_3.0,pyc_6.1,precio_2.0,precio_3.0,precio_6.1
0,2023-01-01,2023,1,ene,1,0,P3,P6,0.00,24.02,...,0.101246,35.041877,35.041877,31.204328,3.175,2.37,1.426,38.216877,37.411877,32.630328
1,2023-01-01,2023,1,ene,1,1,P3,P6,0.00,24.80,...,0.105451,36.307322,36.307322,32.198686,3.175,2.37,1.426,39.482322,38.677322,33.624686
2,2023-01-01,2023,1,ene,1,2,P3,P6,0.00,36.09,...,0.111556,51.170703,51.170703,45.114204,3.175,2.37,1.426,54.345703,53.540703,46.540204
3,2023-01-01,2023,1,ene,1,3,P3,P6,0.00,36.05,...,0.116196,51.559309,51.559309,45.257209,3.175,2.37,1.426,54.734309,53.929309,46.683209
4,2023-01-01,2023,1,ene,1,4,P3,P6,0.00,37.79,...,0.119037,54.08609,54.08609,47.348712,3.175,2.37,1.426,57.26109,56.45609,48.774712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19099,2025-03-06,2025,3,mar,6,19,P1,P2,126.02,2.66,...,0.082511,162.879241,163.803858,145.929771,92.539,36.409,24.986,255.418241,200.212858,170.915771
19100,2025-03-06,2025,3,mar,6,20,P1,P2,117.01,3.15,...,0.081998,152.319422,153.152869,136.503746,92.539,36.409,24.986,244.858422,189.561869,161.489746
19101,2025-03-06,2025,3,mar,6,21,P1,P2,105.51,3.09,...,0.084072,138.809001,139.555604,124.045466,92.539,36.409,24.986,231.348001,175.964604,149.031466
19102,2025-03-06,2025,3,mar,6,22,P2,P3,91.96,4.34,...,0.086071,123.839028,124.369732,110.6325,28.201,17.674,11.795,152.040028,142.043732,122.4275


In [644]:
df_datos['coste_2.0'] = (df_datos['spot'] + df_datos['ssaa'] + df_datos['osom'] + df_datos['ppcc_2.0'] + df_datos['otros']) * 1.015 * (1 + df_datos['perd_2.0'])
df_datos['coste_3.0'] = (df_datos['spot'] + df_datos['ssaa'] + df_datos['osom'] + df_datos['ppcc_3.0'] + df_datos['otros']) * 1.015 * (1 + df_datos['perd_3.0'])
df_datos['coste_6.1'] = (df_datos['spot'] + df_datos['ssaa'] + df_datos['osom'] + df_datos['ppcc_6.1'] + df_datos['otros']) * 1.015 * (1 + df_datos['perd_6.1'])

In [645]:
df_datos['pyc_2.0'] = None
for _, row in df_pycs_energia.iterrows():
    mask = (df_datos["fecha"] >= row["fecha_inicio"]) & (df_datos["fecha"] <= row["fecha_final"]) & (df_datos["dh_3p"] == row["periodo"])
    df_datos.loc[mask, "pyc_2.0"] = row["2.0TD"]
df_datos['pyc_3.0'] = None
for _, row in df_pycs_energia.iterrows():
    mask = (df_datos["fecha"] >= row["fecha_inicio"]) & (df_datos["fecha"] <= row["fecha_final"]) & (df_datos["dh_6p"] == row["periodo"])
    df_datos.loc[mask, "pyc_3.0"] = row["3.0TD"]
df_datos['pyc_6.1'] = None    
for _, row in df_pycs_energia.iterrows():
    mask = (df_datos["fecha"] >= row["fecha_inicio"]) & (df_datos["fecha"] <= row["fecha_final"]) & (df_datos["dh_6p"] == row["periodo"])
    df_datos.loc[mask, "pyc_6.1"] = row["6.1TD"]  
df_datos[['pyc_2.0', 'pyc_3.0', 'pyc_6.1']] *= 1000

In [646]:
df_datos['precio_2.0'] = df_datos['coste_2.0'] + df_datos['pyc_2.0']
df_datos['precio_3.0'] = df_datos['coste_3.0'] + df_datos['pyc_3.0']
df_datos['precio_6.1'] = df_datos['coste_6.1'] + df_datos['pyc_6.1']

In [647]:
df_datos

,fecha,año,mes,mes_nombre,dia,hora,dh_3p,dh_6p,spot,ssaa,...,perd_6.1,coste_2.0,coste_3.0,coste_6.1,pyc_2.0,pyc_3.0,pyc_6.1,precio_2.0,precio_3.0,precio_6.1
0,2023-01-01,2023,1,ene,1,0,P3,P6,0.00,24.02,...,0.101246,35.041877,35.041877,31.204328,3.175,2.37,1.426,38.216877,37.411877,32.630328
1,2023-01-01,2023,1,ene,1,1,P3,P6,0.00,24.80,...,0.105451,36.307322,36.307322,32.198686,3.175,2.37,1.426,39.482322,38.677322,33.624686
2,2023-01-01,2023,1,ene,1,2,P3,P6,0.00,36.09,...,0.111556,51.170703,51.170703,45.114204,3.175,2.37,1.426,54.345703,53.540703,46.540204
3,2023-01-01,2023,1,ene,1,3,P3,P6,0.00,36.05,...,0.116196,51.559309,51.559309,45.257209,3.175,2.37,1.426,54.734309,53.929309,46.683209
4,2023-01-01,2023,1,ene,1,4,P3,P6,0.00,37.79,...,0.119037,54.08609,54.08609,47.348712,3.175,2.37,1.426,57.26109,56.45609,48.774712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19099,2025-03-06,2025,3,mar,6,19,P1,P2,126.02,2.66,...,0.082511,162.879241,163.803858,145.929771,92.539,36.409,24.986,255.418241,200.212858,170.915771
19100,2025-03-06,2025,3,mar,6,20,P1,P2,117.01,3.15,...,0.081998,152.319422,153.152869,136.503746,92.539,36.409,24.986,244.858422,189.561869,161.489746
19101,2025-03-06,2025,3,mar,6,21,P1,P2,105.51,3.09,...,0.084072,138.809001,139.555604,124.045466,92.539,36.409,24.986,231.348001,175.964604,149.031466
19102,2025-03-06,2025,3,mar,6,22,P2,P3,91.96,4.34,...,0.086071,123.839028,124.369732,110.6325,28.201,17.674,11.795,152.040028,142.043732,122.4275


In [648]:
df_datos.to_excel('data_2.xlsx', index = False)